In [1]:
import requests
import time

CSV_URL = "https://data.cityofnewyork.us/api/v3/views/h9gi-nx95/query.csv"
FILENAME = "nyc_vehicle_collisions.csv"

APP_TOKEN = "MljgOH2gik2nDljXO6HqNPeXL"

def download_csv_stream(url, filename, app_token=None):
    print(f"-> İndirme başlatılıyor: {filename}")
    print("-> Bu işlem dosya boyutuna bağlı olarak zaman alabilir...")

    headers = {}
    if app_token:
        headers['X-App-Token'] = app_token

    try:
        with requests.get(url, headers=headers, stream=True) as response:
            response.raise_for_status()

            with open(filename, 'wb') as f:
                total_downloaded = 0
                chunk_size = 8192

                start_time = time.time()

                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        total_downloaded += len(chunk)

                        if total_downloaded % (5 * 1024 * 1024) < chunk_size:
                            mb_size = total_downloaded / (1024 * 1024)
                            print(f"-> İndiriliyor... Şu anki boyut: {mb_size:.2f} MB")

        end_time = time.time()
        print(f"\n İndirme tamamlandı!")
        print(f"-> Toplam Boyut: {total_downloaded / (1024 * 1024):.2f} MB")
        print(f"-> Geçen Süre: {end_time - start_time:.2f} saniye")
        print(f"-> Dosya şuraya kaydedildi: {filename}")

    except requests.exceptions.HTTPError as err:
        print(f"\n Bir hata oluştu (HTTP Hatası): {err}")
        if err.response.status_code == 403:
            print("İPUCU: SODA3 API'si kimlik doğrulama istiyor olabilir. Bir 'App Token' alıp koda eklemeyi deneyin.")
    except Exception as e:
        print(f"\n Beklenmeyen bir hata oluştu: {e}")

download_csv_stream(CSV_URL, FILENAME, APP_TOKEN)

-> İndirme başlatılıyor: nyc_vehicle_collisions.csv
-> Bu işlem dosya boyutuna bağlı olarak zaman alabilir...
-> İndiriliyor... Şu anki boyut: 10.00 MB
-> İndiriliyor... Şu anki boyut: 30.00 MB
-> İndiriliyor... Şu anki boyut: 50.01 MB
-> İndiriliyor... Şu anki boyut: 60.00 MB
-> İndiriliyor... Şu anki boyut: 70.00 MB
-> İndiriliyor... Şu anki boyut: 80.00 MB
-> İndiriliyor... Şu anki boyut: 90.00 MB
-> İndiriliyor... Şu anki boyut: 110.01 MB
-> İndiriliyor... Şu anki boyut: 115.00 MB
-> İndiriliyor... Şu anki boyut: 145.01 MB
-> İndiriliyor... Şu anki boyut: 150.00 MB
-> İndiriliyor... Şu anki boyut: 215.00 MB
-> İndiriliyor... Şu anki boyut: 250.01 MB
-> İndiriliyor... Şu anki boyut: 255.01 MB
-> İndiriliyor... Şu anki boyut: 260.00 MB
-> İndiriliyor... Şu anki boyut: 350.01 MB
-> İndiriliyor... Şu anki boyut: 355.00 MB
-> İndiriliyor... Şu anki boyut: 365.00 MB
-> İndiriliyor... Şu anki boyut: 375.00 MB
-> İndiriliyor... Şu anki boyut: 390.00 MB
-> İndiriliyor... Şu anki boyut: 400.

In [2]:
import pandas as pd

df = pd.read_csv('nyc_vehicle_collisions.csv' , low_memory=False)

print(df.head())

                  :id           :version               :created_at  \
0  row-rsc3.wgwr-zpi9  rv-qukg_54rg~e45q  2021-09-14T22:52:22.882Z   
1  row-t9dc~q8a7~jwk4  rv-vmia.j6tc.thwm  2022-03-29T22:52:48.720Z   
2  row-h8ny_qxbv_ezwp  rv-x4h8~yx6x.vdk4  2023-11-06T23:52:10.201Z   
3  row-gxrs-hm78-w9gs  rv-fazg.rwsa.9gjd  2022-07-04T22:51:56.069Z   
4  row-58i4.zk9q.sxqs  rv-4bks~69au~m7kz  2022-09-26T22:52:05.596Z   

                :updated_at               crash_date crash_time   borough  \
0  2021-09-14T22:52:26.672Z  2021-09-11T00:00:00.000       2:39       NaN   
1  2022-03-29T22:52:49.994Z  2022-03-26T00:00:00.000      11:45       NaN   
2  2023-11-06T23:52:18.104Z  2023-11-01T00:00:00.000       1:29  BROOKLYN   
3  2022-07-04T22:52:04.895Z  2022-06-29T00:00:00.000       6:55       NaN   
4  2022-09-26T22:52:11.092Z  2022-09-21T00:00:00.000      13:21       NaN   

  zip_code  latitude  longitude  ... contributing_factor_vehicle_2  \
0      NaN       NaN        NaN  ...          

In [3]:
df.dropna(subset=['crash_time', 'borough'], inplace=True)

print(f"Temizlik sonrası kalan satır sayısı: {len(df)}")

Temizlik sonrası kalan satır sayısı: 1543364


In [4]:
cols_to_check = ['vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']
df = df[df[cols_to_check].isnull().all(axis=1)]

cols_to_drop = [
    'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5',
    'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5'
]
df.drop(columns=cols_to_drop, inplace=True)

print(f"Filtreleme sonrası kalan satır sayısı: {df.shape[0]}")
print(f"Kalan sütunlar: {df.columns.tolist()}")

Filtreleme sonrası kalan satır sayısı: 1449562
Kalan sütunlar: [':id', ':version', ':created_at', ':updated_at', 'crash_date', 'crash_time', 'borough', 'zip_code', 'latitude', 'longitude', 'location', 'on_street_name', 'off_street_name', 'cross_street_name', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'collision_id', 'vehicle_type_code1', 'vehicle_type_code2']


In [5]:
extra_cols_to_drop = ['zip_code', 'latitude', 'longitude', 'location' , 'on_street_name' , 'cross_street_name' , 'off_street_name']

df.drop(columns=extra_cols_to_drop, inplace=True, errors='ignore')

print(f"Kalan sütun sayısı: {df.shape[1]}")
print("Güncel Sütun Listesi:", df.columns.tolist())

Kalan sütun sayısı: 20
Güncel Sütun Listesi: [':id', ':version', ':created_at', ':updated_at', 'crash_date', 'crash_time', 'borough', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'collision_id', 'vehicle_type_code1', 'vehicle_type_code2']


In [6]:
df.to_csv('nyc_collisions_crashes_cleaned.csv', index=False)

print("Veri 'temizlenmis_nyc_kazalar.csv' adıyla başarıyla kaydedildi!")

Veri 'temizlenmis_nyc_kazalar.csv' adıyla başarıyla kaydedildi!


In [7]:
target_columns = [':id', ':version', ':created_at', ':updated_at', 'crash_date']

df.dropna(subset=target_columns, inplace=True)

print(f"Temizlik sonrası kalan toplam satır sayısı: {len(df)}")
print("\nSeçilen kolonlardaki güncel boş değer sayıları:")
print(df[target_columns].isnull().sum())

Temizlik sonrası kalan toplam satır sayısı: 1449562

Seçilen kolonlardaki güncel boş değer sayıları:
:id            0
:version       0
:created_at    0
:updated_at    0
crash_date     0
dtype: int64


In [8]:
severity_columns = [
    'number_of_persons_injured',
    'number_of_persons_killed',
    'number_of_pedestrians_injured',
    'number_of_pedestrians_killed',
    'number_of_cyclist_injured',
    'number_of_cyclist_killed',
    'number_of_motorist_injured',
    'number_of_motorist_killed'
]

df.dropna(subset=severity_columns, inplace=True)

print(f"Sayısal temizlik sonrası kalan satır sayısı: {len(df)}")

Sayısal temizlik sonrası kalan satır sayısı: 1449536


In [9]:
target_column = ['collision_id']

df.dropna(subset=target_column, inplace=True)

print(f"Temizlik sonrası kalan toplam satır sayısı: {len(df)}")
print("\nSeçilen kolonlardaki güncel boş değer sayıları:")
print(df[target_column].isnull().sum())

Temizlik sonrası kalan toplam satır sayısı: 1449536

Seçilen kolonlardaki güncel boş değer sayıları:
collision_id    0
dtype: int64


In [10]:
df.dropna(subset=['contributing_factor_vehicle_1', 'vehicle_type_code1'], inplace=True)

print(f"Son temizlik sonrası kalan satır sayısı: {len(df)}")
print(f"Kalan Null Değer Sayısı (Factor 1): {df['contributing_factor_vehicle_1'].isnull().sum()}")
print(f"Kalan Null Değer Sayısı (Vehicle Type 1): {df['vehicle_type_code1'].isnull().sum()}")

Son temizlik sonrası kalan satır sayısı: 1434710
Kalan Null Değer Sayısı (Factor 1): 0
Kalan Null Değer Sayısı (Vehicle Type 1): 0


In [11]:
df['contributing_factor_vehicle_2'] = df['contributing_factor_vehicle_2'].fillna('Not Applicable')
df['vehicle_type_code2'] = df['vehicle_type_code2'].fillna('Not Applicable')

In [12]:
df.to_csv('nyc_collisions_crashes_cleaned.csv', index=False)

print("Veri 'temizlenmis_nyc_kazalar.csv' adıyla başarıyla kaydedildi!")

Veri 'temizlenmis_nyc_kazalar.csv' adıyla başarıyla kaydedildi!


In [2]:
import pandas as pd

df = pd.read_csv('nyc_collisions_crashes_cleaned.csv')

print(f"Kalan sütun sayısı: {df.shape[1]}")
print("Güncel Sütun Listesi:", df.columns.tolist())

Kalan sütun sayısı: 20
Güncel Sütun Listesi: [':id', ':version', ':created_at', ':updated_at', 'crash_date', 'crash_time', 'borough', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'collision_id', 'vehicle_type_code1', 'vehicle_type_code2']


In [3]:
print(f"Sütun 1 Benzersiz Sayı: {df['vehicle_type_code1'].nunique()}")
print(f"Sütun 2 Benzersiz Sayı: {df['vehicle_type_code2'].nunique()}")

all_unique_types = pd.concat([df['vehicle_type_code1'], df['vehicle_type_code2']]).unique()
print(f"\nToplam Ortak Benzersiz Araç Tipi Sayısı: {len(all_unique_types)}")
print("Araç Tipleri Listesi:")
print(all_unique_types)

Sütun 1 Benzersiz Sayı: 1415
Sütun 2 Benzersiz Sayı: 1589

Toplam Ortak Benzersiz Araç Tipi Sayısı: 2314
Araç Tipleri Listesi:
['Sedan' 'Tanker' 'Station Wagon/Sport Utility Vehicle' ... 'RED SCOOTE'
 'TRUCK CRAN' 'util truck']


In [4]:
df['vehicle_type_code1'] = df['vehicle_type_code1'].str.lower().str.strip()
df['vehicle_type_code2'] = df['vehicle_type_code2'].str.lower().str.strip()

all_unique = pd.concat([df['vehicle_type_code1'], df['vehicle_type_code2']]).unique()
print(f"Temizlik sonrası benzersiz sayısı: {len(all_unique)}")

Temizlik sonrası benzersiz sayısı: 1574


In [5]:
all_vehicles = pd.concat([df['vehicle_type_code1'], df['vehicle_type_code2']])
counts_pct = all_vehicles.value_counts(normalize=True) * 100

print("En çok tekrar eden ilk 20 araç tipinin toplam verideki payı (%):")
print(counts_pct.head(20))
print(f"\nİlk 20 tipin toplam kapsayıcılığı: %{counts_pct.head(20).sum():.2f}")

En çok tekrar eden ilk 20 araç tipinin toplam verideki payı (%):
sedan                                  22.349395
passenger vehicle                      17.564386
station wagon/sport utility vehicle    17.506116
not applicable                         10.907535
sport utility / station wagon           7.665591
taxi                                    3.864091
unknown                                 2.791122
pick-up truck                           2.038740
bus                                     1.822319
van                                     1.752375
4 dr sedan                              1.523897
bike                                    1.450955
box truck                               1.314342
other                                   1.158527
small com veh(4 tires)                  0.704498
large com veh(6 or more tires)          0.646995
motorcycle                              0.597891
bicycle                                 0.596218
livery vehicle                          0.510626
trac

In [7]:
df.to_csv('nyc_vehicle_collisions_cleaned.csv', index=False)

print("Veri 'nyc_vehicle_collisions_cleaned.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_cleaned.csv' adıyla başarıyla kaydedildi!


In [8]:
import pandas as pd

def strict_top20_filter(df):
    mapping = {
        'sedan': 'passenger car',
        'passenger vehicle': 'passenger car',
        '4 dr sedan': 'passenger car',

        'station wagon/sport utility vehicle': 'suv/wagon',
        'sport utility / station wagon': 'suv/wagon',

        'taxi': 'public service/taxi',
        'bus': 'public service/taxi',
        'livery vehicle': 'public service/taxi',

        'pick-up truck': 'truck/commercial',
        'box truck': 'truck/commercial',
        'small com veh(4 tires)': 'truck/commercial',
        'large com veh(6 or more tires)': 'truck/commercial',
        'tractor truck diesel': 'truck/commercial',

        'van': 'van',

        'bike': 'two wheeled',
        'motorcycle': 'two wheeled',
        'bicycle': 'two wheeled',

        'not applicable': 'not applicable',
        'unknown': 'unknown',
        'other': 'other'
    }

    initial_count = len(df)

    df['vehicle_type_code1'] = df['vehicle_type_code1'].astype(str).str.lower().str.strip().map(mapping)
    df['vehicle_type_code2'] = df['vehicle_type_code2'].astype(str).str.lower().str.strip().map(mapping)

    df = df.dropna(subset=['vehicle_type_code1', 'vehicle_type_code2'])

    print(f"Filtreleme tamamlandı.")
    print(f"Top 20 dışı olduğu için silinen satır sayısı: {initial_count - len(df)}")
    print(f"Kalan temiz satır sayısı: {len(df)}")

    return df

df = strict_top20_filter(df)

print("\nKalan Benzersiz Kategoriler:")
print(pd.concat([df['vehicle_type_code1'], df['vehicle_type_code2']]).unique())

Filtreleme tamamlandı.
Top 20 dışı olduğu için silinen satır sayısı: 80118
Kalan temiz satır sayısı: 1354592

Kalan Benzersiz Kategoriler:
['passenger car' 'suv/wagon' 'public service/taxi' 'two wheeled'
 'truck/commercial' 'van' 'unknown' 'other' 'not applicable']


In [9]:
df.to_csv('nyc_vehicle_collisions_merged.csv', index=False)

print("Veri 'nyc_vehicle_collisions_merged.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_merged.csv' adıyla başarıyla kaydedildi!


In [11]:
import pandas as pd

df['contributing_factor_vehicle_1'] = df['contributing_factor_vehicle_1'].astype(str).str.lower().str.strip()
df['contributing_factor_vehicle_2'] = df['contributing_factor_vehicle_2'].astype(str).str.lower().str.strip()

all_factors = pd.concat([df['contributing_factor_vehicle_1'], df['contributing_factor_vehicle_2']]).unique()

all_factors.sort()

print(f"Sütun 1 Benzersiz Neden Sayısı: {df['contributing_factor_vehicle_1'].nunique()}")
print(f"Sütun 2 Benzersiz Neden Sayısı: {df['contributing_factor_vehicle_2'].nunique()}")
print(f"Toplam Ortak Benzersiz Neden Sayısı: {len(all_factors)}")

print("\n--- Tüm Kaza Nedenleri Listesi (Temizlenmiş) ---")
for factor in all_factors:
    print(factor)

factors_df = pd.DataFrame(all_factors, columns=['Contributing_Factor_Unique_Values'])
factors_df.to_csv('kaza_nedenleri_unique.csv', index=False, encoding='utf-8-sig')
print("\nTüm benzersiz nedenler 'kaza_nedenleri_unique.csv' dosyasına kaydedildi.")

Sütun 1 Benzersiz Neden Sayısı: 59
Sütun 2 Benzersiz Neden Sayısı: 60
Toplam Ortak Benzersiz Neden Sayısı: 60

--- Tüm Kaza Nedenleri Listesi (Temizlenmiş) ---
1
80
accelerator defective
aggressive driving/road rage
alcohol involvement
animals action
backing unsafely
brakes defective
cell phone (hand-held)
cell phone (hands-free)
driver inattention/distraction
driver inexperience
driverless/runaway vehicle
drugs (illegal)
eating or drinking
failure to keep right
failure to yield right-of-way
fatigued/drowsy
fell asleep
following too closely
glare
headlights defective
illnes
illness
lane marking improper/inadequate
listening/using headphones
lost consciousness
not applicable
obstruction/debris
other electronic device
other lighting defects
other vehicular
outside car distraction
oversized vehicle
passenger distraction
passing or lane usage improper
passing too closely
pavement defective
pavement slippery
pedestrian/bicyclist/other pedestrian error/confusion
physical disability
prescript

In [12]:
def clean_and_filter_factors(df):
    mapping = {
        'driver inattention/distraction': 'distraction',
        'cell phone (hand-held)': 'distraction',
        'cell phone (hands-free)': 'distraction',
        'eating or drinking': 'distraction',
        'listening/using headphones': 'distraction',
        'other electronic device': 'distraction',
        'outside car distraction': 'distraction',
        'passenger distraction': 'distraction',
        'texting': 'distraction',
        'using on board navigation device': 'distraction',

        'aggressive driving/road rage': 'unsafe driving',
        'backing unsafely': 'unsafe driving',
        'failure to keep right': 'unsafe driving',
        'failure to yield right-of-way': 'unsafe driving',
        'following too closely': 'unsafe driving',
        'passing too closely': 'unsafe driving',
        'passing or lane usage improper': 'unsafe driving',
        'turning improperly': 'unsafe driving',
        'unsafe lane changing': 'unsafe driving',
        'unsafe speed': 'unsafe driving',
        'traffic control disregarded': 'unsafe driving',

        'alcohol involvement': 'impairment',
        'drugs (illegal)': 'impairment',
        'prescription medication': 'impairment',
        'fatigued/drowsy': 'impairment',
        'fell asleep': 'impairment',
        'illness': 'impairment',
        'illnes': 'impairment',
        'lost consciousness': 'impairment',
        'physical disability': 'impairment',

        'accelerator defective': 'vehicle defect',
        'brakes defective': 'vehicle defect',
        'headlights defective': 'vehicle defect',
        'other lighting defects': 'vehicle defect',
        'steering failure': 'vehicle defect',
        'tire failure/inadequate': 'vehicle defect',
        'tow hitch defective': 'vehicle defect',
        'windshield inadequate': 'vehicle defect',
        'oversized vehicle': 'vehicle defect',
        'tinted windows': 'vehicle defect',
        'other vehicular': 'vehicle defect',

        'animals action': 'environmental',
        'glare': 'environmental',
        'lane marking improper/inadequate': 'environmental',
        'obstruction/debris': 'environmental',
        'pavement defective': 'environmental',
        'pavement slippery': 'environmental',
        'shoulders defective/improper': 'environmental',
        'traffic control device improper/non-working': 'environmental',
        'view obstructed/limited': 'environmental',

        'driver inexperience': 'human error',
        'driverless/runaway vehicle': 'human error',
        'pedestrian/bicyclist/other pedestrian error/confusion': 'human error',
        'reaction to other uninvolved vehicle': 'human error',
        'reaction to uninvolved vehicle': 'human error',
        'vehicle vandalism': 'human error',

        'unspecified': 'unspecified',
        'not applicable': 'not applicable'
    }

    initial_count = len(df)

    df['contributing_factor_vehicle_1'] = df['contributing_factor_vehicle_1'].astype(str).str.lower().str.strip().map(mapping)
    df['contributing_factor_vehicle_2'] = df['contributing_factor_vehicle_2'].astype(str).str.lower().str.strip().map(mapping)

    df = df.dropna(subset=['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2'])

    print(f"Kaza nedenleri temizliği tamamlandı.")
    print(f"Listeye uymayan veya 1-80 gibi hatalı kod içeren {initial_count - len(df)} satır silindi.")
    print(f"Kalan satır sayısı: {len(df)}")

    return df

df = clean_and_filter_factors(df)

print("\n--- Yeni Kaza Nedenleri Grupları ---")
print(df['contributing_factor_vehicle_1'].value_counts())

Kaza nedenleri temizliği tamamlandı.
Listeye uymayan veya 1-80 gibi hatalı kod içeren 67 satır silindi.
Kalan satır sayısı: 1354525

--- Yeni Kaza Nedenleri Grupları ---
contributing_factor_vehicle_1
unspecified       520141
unsafe driving    355091
distraction       278132
impairment         76673
vehicle defect     58472
human error        38234
environmental      27782
Name: count, dtype: int64


In [13]:
df.to_csv('nyc_vehicle_collisions_merged2.csv', index=False)

print("Veri 'nyc_vehicle_collisions_merged2.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_merged2.csv' adıyla başarıyla kaydedildi!


In [1]:
import pandas as pd
df = pd.read_csv('nyc_vehicle_collisions_merged2.csv')

from sklearn.preprocessing import LabelEncoder

all_categories = pd.concat([df['vehicle_type_code1'], df['vehicle_type_code2']]).unique()

le = LabelEncoder()
le.fit(all_categories)

df['vehicle_type_code1'] = le.transform(df['vehicle_type_code1'])
df['vehicle_type_code2'] = le.transform(df['vehicle_type_code2'])

mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Encoding Sözlüğü:", mapping)

Encoding Sözlüğü: {'not applicable': np.int64(0), 'other': np.int64(1), 'passenger car': np.int64(2), 'public service/taxi': np.int64(3), 'suv/wagon': np.int64(4), 'truck/commercial': np.int64(5), 'two wheeled': np.int64(6), 'unknown': np.int64(7), 'van': np.int64(8)}


In [2]:
all_factors = pd.concat([df['contributing_factor_vehicle_1'], df['contributing_factor_vehicle_2']]).unique()

le_factor = LabelEncoder()
le_factor.fit(all_factors)

df['contributing_factor_vehicle_1'] = le_factor.transform(df['contributing_factor_vehicle_1'])
df['contributing_factor_vehicle_2'] = le_factor.transform(df['contributing_factor_vehicle_2'])

factor_mapping = dict(zip(le_factor.classes_, le_factor.transform(le_factor.classes_)))
print("Kaza Nedeni Encoding Sözlüğü:")
print(factor_mapping)

Kaza Nedeni Encoding Sözlüğü:
{'distraction': np.int64(0), 'environmental': np.int64(1), 'human error': np.int64(2), 'impairment': np.int64(3), 'not applicable': np.int64(4), 'unsafe driving': np.int64(5), 'unspecified': np.int64(6), 'vehicle defect': np.int64(7)}


In [3]:
df.to_csv('nyc_vehicle_collisions_encoded.csv', index=False)

print("Veri 'nyc_vehicle_collisions_encoded.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_encoded.csv' adıyla başarıyla kaydedildi!


In [1]:
import pandas as pd

df = pd.read_csv('nyc_vehicle_collisions_encoded.csv')

print("--- İlk 5 Satır ---")
print(df.head())

print("\n--- Veri Seti Bilgisi (info) ---")
df.info()

print("\n--- Eksik Değer Sayıları ---")
print(df.isnull().sum())

print("\n--- İstatistiksel Özet (describe) ---")
print(df.describe())

print("\n--- Veri Boyutu ---")
print(df.shape)

print("\n--- Yinelenen Kayıt Sayısı ---")
print(df.duplicated().sum())

--- İlk 5 Satır ---
                  :id           :version               :created_at  \
0  row-ih2j-xch6_fenf  rv-e9zc.xr5k-68h5  2021-09-14T22:52:22.882Z   
1  row-hpfw.kfmw_56zk  rv-m9du-h9pt~83rq  2021-12-17T23:52:47.817Z   
2  row-mf46.qqxh.jwhb  rv-fjns_aj3x-6vnq  2021-12-17T23:52:47.817Z   
3  row-ymd4_a5r8-fhnf  rv-6cm2_jf7k~x6tu  2021-12-17T23:52:47.817Z   
4  row-f3c2_qzd4_euqz  rv-tpp9.skng~2fmh  2021-12-17T23:52:47.817Z   

                :updated_at               crash_date crash_time    borough  \
0  2021-09-14T22:52:26.672Z  2021-09-11T00:00:00.000       9:35   BROOKLYN   
1  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000       8:17      BRONX   
2  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      21:10   BROOKLYN   
3  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      14:58  MANHATTAN   
4  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      16:50     QUEENS   

   number_of_persons_injured  number_of_persons_killed  \
0                        0.0    

In [2]:
import pandas as pd

df = pd.read_csv('nyc_vehicle_collisions_merged2.csv')

print("--- İlk 5 Satır ---")
print(df.head())

print("\n--- Veri Seti Bilgisi (info) ---")
df.info()

print("\n--- Eksik Değer Sayıları ---")
print(df.isnull().sum())

print("\n--- İstatistiksel Özet (describe) ---")
print(df.describe())

print("\n--- Veri Boyutu ---")
print(df.shape)

print("\n--- Yinelenen Kayıt Sayısı ---")
print(df.duplicated().sum())

--- İlk 5 Satır ---
                  :id           :version               :created_at  \
0  row-ih2j-xch6_fenf  rv-e9zc.xr5k-68h5  2021-09-14T22:52:22.882Z   
1  row-hpfw.kfmw_56zk  rv-m9du-h9pt~83rq  2021-12-17T23:52:47.817Z   
2  row-mf46.qqxh.jwhb  rv-fjns_aj3x-6vnq  2021-12-17T23:52:47.817Z   
3  row-ymd4_a5r8-fhnf  rv-6cm2_jf7k~x6tu  2021-12-17T23:52:47.817Z   
4  row-f3c2_qzd4_euqz  rv-tpp9.skng~2fmh  2021-12-17T23:52:47.817Z   

                :updated_at               crash_date crash_time    borough  \
0  2021-09-14T22:52:26.672Z  2021-09-11T00:00:00.000       9:35   BROOKLYN   
1  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000       8:17      BRONX   
2  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      21:10   BROOKLYN   
3  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      14:58  MANHATTAN   
4  2021-12-17T23:52:52.600Z  2021-12-14T00:00:00.000      16:50     QUEENS   

   number_of_persons_injured  number_of_persons_killed  \
0                        0.0    

In [1]:
import pandas as pd
df = pd.read_csv("nyc_vehicle_collisions_merged2.csv")

In [2]:
df['crash_date'] = pd.to_datetime(df['crash_date'])

unique_years = df['crash_date'].dt.year.unique()

print(unique_years)

[2021 2023 2022 2016 2025 2020 2024 2012 2018 2017 2019 2013 2014 2015]


In [3]:
df['crash_date'] = pd.to_datetime(df['crash_date'])

years_to_keep = [2021, 2022, 2023, 2024, 2025]

df = df[df['crash_date'].dt.year.isin(years_to_keep)]

print(df['crash_date'].dt.year.unique())

[2021 2023 2022 2025 2024]


In [4]:
df.to_csv('nyc_vehicle_collisions_merged2.csv', index=False)

print("Veri 'nyc_vehicle_collisions_merged2.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_merged2.csv' adıyla başarıyla kaydedildi!


In [5]:
df = pd.read_csv("nyc_vehicle_collisions_encoded.csv")

df['crash_date'] = pd.to_datetime(df['crash_date'])

unique_years = df['crash_date'].dt.year.unique()

print(unique_years)

[2021 2023 2022 2016 2025 2020 2024 2012 2018 2017 2019 2013 2014 2015]


In [6]:
df['crash_date'] = pd.to_datetime(df['crash_date'])

years_to_keep = [2021, 2022, 2023, 2024, 2025]

df = df[df['crash_date'].dt.year.isin(years_to_keep)]

print(df['crash_date'].dt.year.unique())

[2021 2023 2022 2025 2024]


In [7]:
df.to_csv('nyc_vehicle_collisions_encoded.csv', index=False)

print("Veri 'nyc_vehicle_collisions_encoded.csv' adıyla başarıyla kaydedildi!")

Veri 'nyc_vehicle_collisions_encoded.csv' adıyla başarıyla kaydedildi!
